# “大表”制作

In [1]:
import numpy as np
import pandas as pd

import os

基础：

In [2]:
os.chdir(r'D:\paper\rowdata')

In [3]:
code_ls = []
with open("Code.txt", 'r') as code:
    for line in code:
        code_ls.append(line[:-1])

date_ls = ['2009-12-31', '2010-12-31', '2011-12-31',
          '2012-12-31', '2013-12-31', '2014-12-31',
          '2015-12-31', '2016-12-31', '2017-12-31',
          '2018-12-31', '2019-12-31', '2020-12-31',
          '2021-12-31',]

n_1 = len(code_ls)
n_2 = len(date_ls)

Code_ls = []
for code in code_ls:
    for i in range(n_2):
        Code_ls.append(code)

Time_ls = []
for i in range(n_1):
    for date in date_ls:
        Time_ls.append(date)

df = pd.DataFrame({"Code": Code_ls, "Time": Time_ls})
df

,Code,Time
0,000002,2009-12-31
1,000002,2010-12-31
2,000002,2011-12-31
3,000002,2012-12-31
4,000002,2013-12-31
...,...,...
38683,605599,2017-12-31
38684,605599,2018-12-31
38685,605599,2019-12-31
38686,605599,2020-12-31


In [4]:
df["Time"] = df["Time"].astype('datetime64')

# 被解释变量

## 违规与否和违规数量

In [5]:
os.chdir(r'D:\paper\rowdata\violate')

violate = pd.read_csv('violate_clean.csv', parse_dates=[1],
                      infer_datetime_format=True, dtype={"Code": str})
violate = violate.rename(columns={"Time_a": "Time"})

In [6]:
# 将'Y'，'N'转换为1，0
violate["Vio"] = [1 if i == 'Y' else 0 for i in violate["Vio"]]
violate["Vio_wide"] = [1 if i == 'Y' else 0 for i in violate["Vio_wide"]]
df = pd.merge(df, violate, on=['Code', 'Time'], how='outer')

## 其他应收款

In [7]:
os.chdir(r'D:\paper\rowdata\balance')
balance = pd.read_csv("balance_clean.csv", dtype={"Code": str, "Time":str})

qtysk = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][2]
            qtysk.append(q)
        except:
            qtysk.append(np.nan)
df["Qtysk"] = qtysk

qtyfk = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][5]
            qtyfk.append(q)
        except:
            qtyfk.append(np.nan)
df["Qtyfk"] = qtyfk

zcze = []
for item in code_ls:
    for time in date_ls:
        try:
            q = balance[balance["Code"]==item]
            q = q[q["Time"]==time].values[0][4]
            zcze.append(q)
        except:
            zcze.append(np.nan)
df["Zcze"] = zcze
df["Zczelog"] = [np.log(i) if i != 0 else np.nan for i in df["Zcze"]]

解释变量，解决

---

# 被解释变量

In [8]:
os.chdir(r'D:\paper\rowdata\rzrq')

rzrq = pd.read_csv('rzrq_clean1.csv', dtype={"Code": str})

def treated_detect(code, df_t):
    try:
        if df_t[df_t["Code"]==code]["Treated"].iloc[0] == 1:
            if df_t[df_t["Code"]==code]["Tc_t"].iloc[0] == 0:
                return 1
            else:
                return 0
        else:
            return 0
    except:
        return 0

df["Treated"] = [treated_detect(i, rzrq) for i in df["Code"]]

def treated_detect_wide(code, df_t):
    try:
        if df_t[df_t["Code"]==code]["Treated"].iloc[0] == 1:
            return 1
        else:
            return 0
    except:
        return 0
    
df["Treated_wide"] = [treated_detect_wide(i, rzrq) for i in df["Code"]]

def post_detect(code, df):
    if df[df["Code"]==code]["Treated"].iloc[0] == 0:
        return [0 for i in range(13)]
    else:
        # 获取入选年度，若入选年度为2010，结果为10，如入选年度为2018，结果为18
        in_year = int(df[df["Code"]==code]["Time"].iloc[0][:4]) - 2000
        if df[df["Code"]==code]["Tc_t"].iloc[0] == 0:
            n_1 = in_year - 9
            n_2 = 13 - n_1
            ls = [0 for i in range(n_1)] + [1 for i in range(n_2)]
        else:
            # 获取被剔除年度
            out_year = int(df[df["Code"]==code]["Time_a"].iloc[0][:4]) - 2000
            n_1 = in_year - 9
            n_2 = out_year - in_year
            n_3 = 21 - out_year + 1
            ls = [0 for i in range(n_1)] + [1 for i in range(n_2)] + [0 for i in range(n_3)]
    return ls

post = []
for code in code_ls:
    post = post + post_detect(code, rzrq)
df["Post"] = post

df["Rzrq"] = df["Treated"] * df["Post"]

## Treated变量

我们需要设计一个函数，来判断证券是否是融资融券标的，思路如下： 
我们最终需要的结果是两列，一列是`Treated`，一列是`Treated_wide`，前者指进入融资融券标的且未被剔除，后者则只需曾经进入过  
1. 若`Treated`是1且`Tc_t`不是1，则可认为是属于`Treated`组的
2. 若`Treated`是1且`Tc_t`是1，则可认为是属于`Treated_wide`组的

## Post变量

思路如下：
1. 如果rzrq中的`Treated`是0，那么直接生成0就好了
2. 如果rzrq中的`Treated`是1，则取`rzrq`中查询其成为标的的时间，以此生成结果举例来说:
    * 000002从2010年被纳入融资融券标的，则生成`[0,1,1,1,1,1,1,1,1,1,1,1,1]`(2009年融资融券制度尚未实行，之后是1)  
    * 000008自2018年被纳入融资融券标的，则生成`[0,0,0,0,0,0,0,0,0,1,1,1,1]`
    * 000410自2014年被纳入标的，但在2017年被剔除，则生成`[0,0,0,0,0,1,1,1,0,0,0,0,0]`

构建Post列

## 融资交易、融券交易

现有文献的主流方法是亦融资买入额与流通市值的比值、融券卖出额与流通市值的比值、融资余额与流通市值的比值，融券余额与流通市值的比值来表示、分离融资、融券的效果. 我们当然可以直接将这样的数据计算出来在加入到“大表”中，但考虑到有些文献有不同的处理方式，所以我们就将原始数据保存：

In [9]:
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,Treated,Treated_wide,Post,Rzrq
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,1,1,0,0
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,1,1,1,1
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,1,1,1,1
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,1,1,1,1
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0


In [11]:
'''
df["Time"] = df["Time"].astype(str)

os.chdir(r'D:\paper\rowdata\rzrq_v')

rzrq_j = pd.read_csv("rzrq_j_clean.csv", dtype={"Stockcode": str})
rzrq_j = rzrq_j.rename(columns={"Stockcode": "Code"})

rzmr = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            rzmr.append(p)
        except:
            rzmr.append(0)
df["Rzmr"] = rzmr
'''
rzye = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            rzye.append(p)
        except:
            rzye.append(0)
df["Rzye"] = rzye
'''
rqmc = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            rqmc.append(p)
        except:
            rqmc.append(0)
df["Rqmc"] = rqmc

rqye = []
for code in code_ls:
    for date in date_ls:
        try:
            q = rzrq_j.loc[rzrq_j["Year"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            rqye.append(p)
        except:
            rqye.append(0)
df["Rqye"] = rqye
'''
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,Treated,Treated_wide,Post,Rzrq,Rzmr,Rzye,Rqmc,Rqye
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,1,1,0,0,0,0,0,0
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,1,1,1,1,809024122,169648899,2470801,350994
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,1,1,1,1,3430765420,557538781,89010446,14947493
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,1,1,1,1,3581010637,553109519,445081266,110673543
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,1,1,1,1,19963681095,2126345912,1072559193,30995261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [13]:
df[df["Code"]=='600820'].iloc[:, [0,1,14,15,16,17]]

,Code,Time,Rzmr,Rzye,Rqmc,Rqye
26845,600820,2009-12-31,0,0,0,0
26846,600820,2010-12-31,0,0,0,0
26847,600820,2011-12-31,0,0,0,0
26848,600820,2012-12-31,0,0,0,0
26849,600820,2013-12-31,2323886111,244302142,3543734,227657
26850,600820,2014-12-31,15935763831,1480274276,38013136,412174
26851,600820,2015-12-31,63738174337,1947434457,66308795,0
26852,600820,2016-12-31,14410566770,1257549480,7752610,2935266
26853,600820,2017-12-31,9997376289,1167176146,5256624,3344
26854,600820,2018-12-31,2788466385,740218530,5552501,350560


---

## 流通市值

In [16]:
os.chdir(r'D:\paper\rowdata\ltsz')
df["Time"] = df["Time"].astype('datetime64')
ltsz = pd.read_csv('ltsz_clean.csv', parse_dates=[1],
                  dtype={"Code": str})
ltsz = ltsz.rename(columns={"ltsz": "Ltsz"})
df = pd.merge(df, ltsz, on=['Code', 'Time'], how='outer')
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,Treated,Treated_wide,Post,Rzrq,Rzmr,Rzye,Rqmc,Rqye,Ltsz
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,1,1,0,0,0,0,0,0,1.046440e+11
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,1,1,1,1,809024122,169648899,2470801,350994,7.957169e+10
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,1,1,1,1,3430765420,557538781,89010446,14947493,7.231150e+10
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,1,1,1,1,3581010637,553109519,445081266,110673543,9.796418e+10
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,1,1,1,1,19963681095,2126345912,1072559193,30995261,7.789111e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN


In [17]:
df["Ltszlog"] = [np.log(i) if i!=0 else np.nan for i in df["Ltsz"]]
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,Treated,Treated_wide,Post,Rzrq,Rzmr,Rzye,Rqmc,Rqye,Ltsz,Ltszlog
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,1,1,0,0,0,0,0,0,1.046440e+11,25.373830
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,1,1,1,1,809024122,169648899,2470801,350994,7.957169e+10,25.099924
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,1,1,1,1,3430765420,557538781,89010446,14947493,7.231150e+10,25.004249
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,1,1,1,1,3581010637,553109519,445081266,110673543,9.796418e+10,25.307868
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,1,1,1,1,19963681095,2126345912,1072559193,30995261,7.789111e+10,25.078578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN


这个数据我们不用前面的方法，我们使用`pd.merge()`来实现

这些缺失值应该绝大多是不是数据库统计失误，其出现的原因在于有些股票是在2009年以后上市的，为处理方便，我们暂且不把缺失值删掉

---

# 控制变量

## 企业规模

现有文献的普遍做法是使用企业资产的对数值

## 资产负债率

In [18]:
os.chdir(r'D:\paper\rowdata\cwzb')

In [19]:
cwzb = pd.read_csv("cwzb.csv", dtype={"Code":str})

zcfz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            zcfz.append(p)
        except:
            zcfz.append(np.nan)
df["Zcfz"] = [i if i >= 0 else np.nan for i in zcfz]

## ROA

In [23]:
roa = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            roa.append(p)
        except:
            roa.append(np.nan)
df["Roa"] = roa
'''
tobin = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][6]
            tobin.append(p)
        except:
            tobin.append(np.nan)
df["TobinQ"] = tobin

zmszb = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][7]
            zmszb.append(p)
        except:
            zmszb.append(np.nan)
df["Zmszb"] = zmszb

yszz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            yszz.append(p)
        except:
            yszz.append(np.nan)
df["Yszz"] = yszz
'''

'\ntobin = []\nfor code in code_ls:\n    for date in date_ls:\n        try:\n            q = cwzb.loc[cwzb["Time"]==date]\n            p = q.loc[q["Code"]==code].values[0][6]\n            tobin.append(p)\n        except:\n            tobin.append(np.nan)\ndf["TobinQ"] = tobin\n\nzmszb = []\nfor code in code_ls:\n    for date in date_ls:\n        try:\n            q = cwzb.loc[cwzb["Time"]==date]\n            p = q.loc[q["Code"]==code].values[0][7]\n            zmszb.append(p)\n        except:\n            zmszb.append(np.nan)\ndf["Zmszb"] = zmszb\n\nyszz = []\nfor code in code_ls:\n    for date in date_ls:\n        try:\n            q = cwzb.loc[cwzb["Time"]==date]\n            p = q.loc[q["Code"]==code].values[0][4]\n            yszz.append(p)\n        except:\n            yszz.append(np.nan)\ndf["Yszz"] = yszz\n'

## TobinQ

In [21]:
tobin = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][6]
            tobin.append(p)
        except:
            tobin.append(np.nan)
df["TobinQ"] = tobin

zmszb = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][7]
            zmszb.append(p)
        except:
            zmszb.append(np.nan)
df["Zmszb"] = zmszb

yszz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = cwzb.loc[cwzb["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            yszz.append(p)
        except:
            yszz.append(np.nan)
df["Yszz"] = yszz

df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Rzmr,Rzye,Rqmc,Rqye,Ltsz,Ltszlog,Zcfz,TobinQ,Zmszb,Yszz
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,0,0,0,0,1.046440e+11,25.373830,0.526484,1.511156,0.661745,0.330062
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,809024122,169648899,2470801,350994,7.957169e+10,25.099924,0.622241,1.165389,0.858083,0.225685
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,3430765420,557538781,89010446,14947493,7.231150e+10,25.004249,0.615285,1.042680,0.959067,0.118359
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,3581010637,553109519,445081266,110673543,9.796418e+10,25.307868,0.687353,1.076892,0.928598,0.452732
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,19963681095,2126345912,1072559193,30995261,7.789111e+10,25.078578,0.721487,0.968694,1.032318,-0.182800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,0.474435,NaN,NaN,NaN
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,0.472009,NaN,NaN,-0.027633
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,0.452467,NaN,NaN,-0.165505


In [24]:
df[df["Code"]=='000002'].iloc[:, [0,1,20,21,22,23,24]]

,Code,Time,Zcfz,TobinQ,Zmszb,Yszz,Roa
0,000002,2009-12-31,0.526484,1.511156,0.661745,0.330062,0.048002
1,000002,2010-12-31,0.622241,1.165389,0.858083,0.225685,0.042242
2,000002,2011-12-31,0.615285,1.042680,0.959067,0.118359,0.049637
3,000002,2012-12-31,0.687353,1.076892,0.928598,0.452732,0.051440
4,000002,2013-12-31,0.721487,0.968694,1.032318,-0.182800,0.053727
5,000002,2014-12-31,0.695737,1.064914,0.939043,-0.041141,0.070274
6,000002,2015-12-31,0.704438,1.192347,0.838682,0.486476,0.053896
7,000002,2016-12-31,0.717272,1.069010,0.935445,0.060137,0.061561
8,000002,2017-12-31,0.796925,1.117963,0.894484,0.700284,0.048797
9,000002,2018-12-31,0.824461,1.015736,0.984508,0.650771,0.058444


## 账面市值比

## 营收增长率

---

## 回报率

In [25]:
os.chdir(r'D:\paper\rowdata\scsj')

In [26]:
scsj = pd.read_csv("scsj.csv", dtype={"Code": str})
scsj["Time"] = [i[:4] + "-12-31" for i in scsj["Time"]]

hbl = []
for code in code_ls:
    for date in date_ls:
        try:
            q = scsj.loc[scsj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            hbl.append(p)
        except:
            hbl.append(np.nan)
df["Hbl"] = hbl

hsl = []
for code in code_ls:
    for date in date_ls:
        try:
            q = scsj.loc[scsj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            hsl.append(p)
        except:
            hsl.append(np.nan)
df["Hsl"] = hsl

In [27]:
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Rqye,Ltsz,Ltszlog,Zcfz,TobinQ,Zmszb,Yszz,Roa,Hbl,Hsl
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,0,1.046440e+11,25.373830,0.526484,1.511156,0.661745,0.330062,0.048002,0.683490,1.962740
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,350994,7.957169e+10,25.099924,0.622241,1.165389,0.858083,0.225685,0.042242,-0.232341,0.964916
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,14947493,7.231150e+10,25.004249,0.615285,1.042680,0.959067,0.118359,0.049637,-0.079738,0.609973
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,110673543,9.796418e+10,25.307868,0.687353,1.076892,0.928598,0.452732,0.051440,0.373709,0.589176
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,30995261,7.789111e+10,25.078578,0.721487,0.968694,1.032318,-0.182800,0.053727,-0.194315,0.948251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,NaN,NaN,0.474435,NaN,NaN,NaN,NaN,NaN,NaN
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,NaN,NaN,0.472009,NaN,NaN,-0.027633,0.102316,NaN,NaN
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0,NaN,NaN,0.452467,NaN,NaN,-0.165505,0.087727,NaN,NaN


In [29]:
df[df["Code"]=='600367'].iloc[:, [0,1,25,26]]

,Code,Time,Hbl,Hsl
22178,600367,2009-12-31,1.186937,4.951994
22179,600367,2010-12-31,0.799176,6.774349
22180,600367,2011-12-31,-0.291929,2.860218
22181,600367,2012-12-31,-0.171382,1.277021
22182,600367,2013-12-31,-0.172223,1.577943
22183,600367,2014-12-31,0.312732,1.877836
22184,600367,2015-12-31,0.146827,4.463025
22185,600367,2016-12-31,0.059195,4.473254
22186,600367,2017-12-31,-0.154561,2.926256
22187,600367,2018-12-31,-0.366041,1.267387


## 换手率

---

## 第一大股东持股比例

In [30]:
os.chdir(r'D:\paper\rowdata\holdings')
holdings = pd.read_csv('holdings_clean.csv', dtype={"Code": str})
first = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            first.append(p)
        except:
            first.append(np.nan)
first = np.array(first)
first = first / 100
df["First"] = first

jgtzz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][5]
            jgtzz.append(p)
        except:
            jgtzz.append(np.nan)
jgtzz = np.array(jgtzz)
jgtzz = jgtzz / 100
df["Jgtzz"] = jgtzz

ddzb = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][9]
            ddzb.append(p)
        except:
            ddzb.append(np.nan)
ddzb = np.array(ddzb)
ddzb = ddzb / 100
df["Ddzb"] = ddzb

glccg = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][7]
            glccg.append(p)
        except:
            glccg.append(np.nan)
df["Glccg"] = glccg

lzhy = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][6]
            lzhy.append(p)
        except:
            lzhy.append(np.nan)
df["Lzhy"] = lzhy

dshgm = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][8]
            dshgm.append(p)
        except:
            dshgm.append(np.nan)
df["Dshgm"] = dshgm

In [31]:
gqzhd = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][4]
            gqzhd.append(p)
        except:
            gqzhd.append(np.nan)
df["Gqzhd"] = gqzhd

two_ten = []
for code in code_ls:
    for date in date_ls:
        try:
            q = holdings.loc[holdings["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            two_ten.append(p)
        except:
            two_ten.append(np.nan)
df["Two"] = two_ten

In [36]:
df["Two"] = df["Two"] / 100

In [39]:
df[df["Code"]=='000009'].iloc[:,[0,1,27,28,29,30,31,32,33,34]]

,Code,Time,First,Jgtzz,Ddzb,Glccg,Lzhy,Dshgm,Gqzhd,Two
78,000009,2009-12-31,0.084041,0.423149,0.3750,0.0254,1.0,8.0,1.5357,0.185768
79,000009,2010-12-31,0.084367,0.425272,0.3333,0.0173,1.0,9.0,1.8649,0.221495
80,000009,2011-12-31,0.119568,0.376706,0.3333,0.0168,1.0,9.0,1.1672,0.202047
81,000009,2012-12-31,0.123429,0.322472,0.3750,0.0151,1.0,8.0,0.9716,0.167815
82,000009,2013-12-31,0.123429,0.296076,0.3750,0.0151,1.0,8.0,0.9927,0.153038
83,000009,2014-12-31,0.123429,0.265153,0.4286,0.0151,1.0,7.0,0.7674,0.118185
84,000009,2015-12-31,0.119113,0.237610,0.4286,0.0842,1.0,7.0,0.6977,0.114622
85,000009,2016-12-31,0.119113,0.204071,0.3333,0.1022,1.0,9.0,0.6474,0.095072
86,000009,2017-12-31,0.119113,0.204031,0.3333,0.1232,1.0,9.0,0.6490,0.099131
87,000009,2018-12-31,0.119113,0.209145,0.3333,0.1232,1.0,9.0,1.0327,0.153859


## 机构投资者（基金持股比例）

## 独立董事占比

## 管理层持股比例

## 两职合一

## 董事会规模

---

## 分析师追踪人数

## 四大审计机构

In [40]:
os.chdir(r'D:\paper\rowdata\wbhj')
wbhj = pd.read_csv("wbhj.csv", dtype={"Code": str})
fxszz = []
for code in code_ls:
    for date in date_ls:
        try:
            q = wbhj.loc[wbhj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][2]
            fxszz.append(p)
        except:
            fxszz.append(np.nan)
df["Fxszz"] = fxszz

big4 = []
for code in code_ls:
    for date in date_ls:
        try:
            q = wbhj.loc[wbhj["Time"]==date]
            p = q.loc[q["Code"]==code].values[0][3]
            big4.append(p)
        except:
            big4.append(np.nan)
big4_ls = []
for i in big4:
    if i == 1:
        big4_ls.append(1)
    elif i == 2:
        big4_ls.append(0)
    else:
        big4_ls.append(np.nan)

df["Big4"] = big4_ls

In [44]:
df[df["Code"]=='600513'].iloc[:, [0,1,35,36]]

,Code,Time,Fxszz,Big4
23504,600513,2009-12-31,1.0,0.0
23505,600513,2010-12-31,1.0,0.0
23506,600513,2011-12-31,1.0,0.0
23507,600513,2012-12-31,0.0,0.0
23508,600513,2013-12-31,0.0,0.0
23509,600513,2014-12-31,0.0,0.0
23510,600513,2015-12-31,3.0,0.0
23511,600513,2016-12-31,0.0,0.0
23512,600513,2017-12-31,0.0,0.0
23513,600513,2018-12-31,0.0,0.0


---

## 行业

In [45]:
os.chdir(r'D:\paper\rowdata\basics')
basics = pd.read_csv("basics_clean.csv", parse_dates = [3,4], infer_datetime_format=True,
                    dtype={"Code": str})
industry = []
for code in code_ls:
    for date in date_ls:
        try:
            p = basics.loc[basics["Code"]==code].values[0][2]
            industry.append(p[0])
        except:
            industry.append(np.nan)
df["Industry"] = industry

cqxz = []
for code in code_ls:
    for date in date_ls:
        try:
            p = basics.loc[basics["Code"]==code].values[0][7]
            cqxz.append(p)
        except:
            cqxz.append(np.nan)
df["Cqxz"] = [1 if i in ['P0301', 'P0307', 'P0308'] else 0 for i in cqxz]

basics["Estbdt"] = basics["Estbdt"].astype(str)
basics["Listdt"] = basics["Listdt"].astype(str)
qynl = []
for code in code_ls:
    for date in date_ls:
        try:
            p = int(basics.loc[basics["Code"]==code].values[0][3][:4])
            p = int(date[:4]) - p
            qynl.append(p)
        except:
            qynl.append(np.nan)
df["Qynl"] = qynl

ssnx = []
for code in code_ls:
    for date in date_ls:
        try:
            p = int(basics.loc[basics["Code"]==code].values[0][4][:4])
            p = int(date[:4]) - p
            ssnx.append(p)
        except:
            ssnx.append(np.nan)
df["Ssnx"] = ssnx

In [51]:
df[df["Code"]=='600230'].iloc[:, [0,1,37,38,39,40]]

,Code,Time,Industry,Cqxz,Qynl,Ssnx
20761,600230,2009-12-31,C,1,11,9
20762,600230,2010-12-31,C,1,12,10
20763,600230,2011-12-31,C,1,13,11
20764,600230,2012-12-31,C,1,14,12
20765,600230,2013-12-31,C,1,15,13
20766,600230,2014-12-31,C,1,16,14
20767,600230,2015-12-31,C,1,17,15
20768,600230,2016-12-31,C,1,18,16
20769,600230,2017-12-31,C,1,19,17
20770,600230,2018-12-31,C,1,20,18


In [53]:
industry = []
for code in code_ls:
    for date in date_ls:
        try:
            p = basics.loc[basics["Code"]==code].values[0][2]
            industry.append(p)
        except:
            industry.append(np.nan)
df["Industry_d"] = industry
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Dshgm,Gqzhd,Two,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx,Industry_d
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,10.0,0.3123,0.081711,65.0,1.0,K,1,21,18,K70
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,11.0,0.2797,0.080461,63.0,1.0,K,1,22,19,K70
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,11.0,0.3232,0.084157,59.0,1.0,K,1,23,20,K70
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,11.0,0.3159,0.087051,57.0,1.0,K,1,24,21,K70
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,11.0,0.2762,0.075971,53.0,1.0,K,1,25,22,K70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,F,0,23,-4,F52
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,F,0,24,-3,F52
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,F,0,25,-2,F52
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,F,0,26,-1,F52


## 行业TobinQ

In [54]:
os.chdir(r'D:\paper\rowdata')
hyt = pd.read_csv("hyt.csv", dtype={"Time":str})
hyt

,Ind,Time,HTobin
0,D45,2009-12-31,3.398137
1,D46,2009-12-31,1.223553
2,E47,2009-12-31,0.942311
3,E48,2009-12-31,0.565340
4,E49,2009-12-31,1.851209
...,...,...,...
993,H62,2021-12-31,1.598569
994,A05,2021-12-31,2.194876
995,R87,2021-12-31,3.276718
996,A02,2021-12-31,1.518636


In [68]:
#str(df.iloc[1, :].values[1])[:10]
#df.iloc[1, :].values[41]
qq = hyt[hyt["Ind"]=='K70']
qq[qq["Time"]=='2010-12-31'].values[0][2]

1.0592

In [71]:
hytq = []
n = len(df["Code"])
for i in range(n):
    time = str(df.iloc[i, :].values[1])[:10]
    ind = df.iloc[i, :].values[41]
    try:
        qq = hyt[hyt["Ind"]==ind]
        qq = qq[qq["Time"]==time].values[0][2]
        hytq.append(qq)
    except:
        hytq.append(np.nan)

df["HTobinQ"] = hytq
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Gqzhd,Two,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx,Industry_d,HTobinQ
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,0.3123,0.081711,65.0,1.0,K,1,21,18,K70,1.293807
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,0.2797,0.080461,63.0,1.0,K,1,22,19,K70,1.059200
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.3232,0.084157,59.0,1.0,K,1,23,20,K70,0.626153
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.3159,0.087051,57.0,1.0,K,1,24,21,K70,0.668239
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.2762,0.075971,53.0,1.0,K,1,25,22,K70,0.712191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,F,0,23,-4,F52,1.397437
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,F,0,24,-3,F52,0.936766
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,F,0,25,-2,F52,1.110823
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,F,0,26,-1,F52,1.280476


In [76]:
os.chdir(r'D:\paper\rowdata')
hhi = pd.read_csv("HHI_clean.csv", dtype={"Time":str})
hhi

,Ind,Time,HHI
0,A01,2009-12-31,0.156397
1,A02,2009-12-31,0.543459
2,A03,2009-12-31,0.188079
3,A04,2009-12-31,0.238033
4,A05,2009-12-31,0.926152
...,...,...,...
993,H62,2021-12-31,0.524013
994,A05,2021-12-31,NaN
995,R87,2021-12-31,0.414917
996,A02,2021-12-31,1.000000


In [77]:
hhi_ls = []
n = len(df["Code"])
for i in range(n):
    time = str(df.iloc[i, :].values[1])[:10]
    ind = df.iloc[i, :].values[41]
    try:
        qq = hhi[hhi["Ind"]==ind]
        qq = qq[qq["Time"]==time].values[0][2]
        hhi_ls.append(qq)
    except:
        hhi_ls.append(np.nan)

df["HHI"] = hhi_ls
df

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Two,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx,Industry_d,HTobinQ,HHI
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,0.081711,65.0,1.0,K,1,21,18,K70,1.293807,0.075733
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,0.080461,63.0,1.0,K,1,22,19,K70,1.059200,0.054805
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.084157,59.0,1.0,K,1,23,20,K70,0.626153,0.063567
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.087051,57.0,1.0,K,1,24,21,K70,0.668239,0.071580
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.075971,53.0,1.0,K,1,25,22,K70,0.712191,0.076325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,23,-4,F52,1.397437,0.073352
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,24,-3,F52,0.936766,0.092568
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,25,-2,F52,1.110823,0.093244
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,26,-1,F52,1.280476,0.131845


## 产权性质

## 企业年龄

## 上市年限

---

搞定！

看一下没有缺失数据有多少条数据：

In [52]:
q = df[df["Time"]>'2010']
q.dropna()

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Lzhy,Dshgm,Gqzhd,Two,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,0.0,11.0,0.2797,0.080461,63.0,1.0,K,1,22,19
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.0,11.0,0.3232,0.084157,59.0,1.0,K,1,23,20
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.0,11.0,0.3159,0.087051,57.0,1.0,K,1,24,21
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.0,11.0,0.2762,0.075971,53.0,1.0,K,1,25,22
5,000002,2014-12-31,0,0,0,0,1.225832e+11,5.042067e+10,1.746387e+11,25.885985,...,0.0,11.0,1.2515,0.229777,38.0,1.0,K,1,26,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38440,605376,2021-12-31,0,0,0,0,4.211488e+08,5.020814e+07,1.592191e+09,21.188377,...,0.0,9.0,1.5282,0.424635,6.0,0.0,C,0,11,1
38453,605377,2021-12-31,0,0,0,0,3.266208e+08,1.432862e+07,3.131245e+09,21.864696,...,0.0,9.0,0.8887,0.320369,4.0,0.0,C,0,12,1
38479,605388,2021-12-31,0,1,0,1,2.775852e+08,1.695521e+06,1.921355e+09,21.376297,...,0.0,10.0,1.2345,0.465966,1.0,0.0,C,0,23,1
38518,605399,2021-12-31,0,0,0,0,3.573900e+06,1.390492e+07,1.823445e+09,21.323993,...,0.0,7.0,1.2491,0.426052,1.0,0.0,C,0,15,1


22516 够用了

In [281]:
os.chdir(r'D:\paper\rowdata')

In [282]:
df.to_csv("row_data.csv", index=False)

In [283]:
q = df.dropna()
q.to_csv("row_data_clean_2009.csv", index=False)

In [284]:
qq = df.dropna()
qq = qq[qq["Time"]>'2010']
qq

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Treated,Treated_wide,Post,Rzmr,...,Ddzb,Glccg,Lzhy,Dshgm,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx
1,000002,2010-12-31,0,0,0,0,1,1,1,809024122,...,0.3636,0.2348,0.0,11.0,63.0,1.0,K70,1,22,19
2,000002,2011-12-31,0,0,0,0,1,1,1,3430765420,...,0.3636,0.2209,0.0,11.0,59.0,1.0,K70,1,23,20
3,000002,2012-12-31,0,0,0,0,1,1,1,3581010637,...,0.3636,0.2142,0.0,11.0,57.0,1.0,K70,1,24,21
4,000002,2013-12-31,0,0,0,0,1,1,1,19963681095,...,0.3636,0.2884,0.0,11.0,53.0,1.0,K70,1,25,22
5,000002,2014-12-31,0,0,0,0,1,1,1,31483716925,...,0.3636,0.1815,0.0,11.0,38.0,1.0,K70,1,26,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38440,605376,2021-12-31,0,0,0,0,0,0,0,0,...,0.3333,6.4982,0.0,9.0,6.0,0.0,C33,0,11,1
38453,605377,2021-12-31,0,0,0,0,0,0,0,0,...,0.3333,22.8094,0.0,9.0,4.0,0.0,C22,0,12,1
38479,605388,2021-12-31,0,1,0,1,0,0,0,0,...,0.3000,34.4222,0.0,10.0,1.0,0.0,C15,0,23,1
38518,605399,2021-12-31,0,0,0,0,0,0,0,0,...,0.4286,66.6472,0.0,7.0,1.0,0.0,C26,0,15,1


In [285]:
qq.to_csv("row_data_clean.csv", index=False)

In [81]:
row_data = df
row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Two,Fxszz,Big4,Industry,Cqxz,Qynl,Ssnx,Industry_d,HTobinQ,HHI
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,0.081711,65.0,1.0,K,1,21,18,K70,1.293807,0.075733
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,0.080461,63.0,1.0,K,1,22,19,K70,1.059200,0.054805
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.084157,59.0,1.0,K,1,23,20,K70,0.626153,0.063567
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.087051,57.0,1.0,K,1,24,21,K70,0.668239,0.071580
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.075971,53.0,1.0,K,1,25,22,K70,0.712191,0.076325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38683,605599,2017-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,23,-4,F52,1.397437,0.073352
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,24,-3,F52,0.936766,0.092568
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,25,-2,F52,1.110823,0.093244
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,F,0,26,-1,F52,1.280476,0.131845


In [82]:
row_data["Dshgmlog"] = [np.log(i) if i > 0 else np.nan for i in row_data["Dshgm"]]
row_data["Dshgmlog"]

0        2.302585
1        2.397895
2        2.397895
3        2.397895
4        2.397895
           ...   
38683         NaN
38684         NaN
38685         NaN
38686         NaN
38687    2.197225
Name: Dshgmlog, Length: 38688, dtype: float64

In [83]:
row_data.columns

Index(['Code', 'Time', 'Vio', 'Vio_wide', 'NumVio', 'NumVio_wide', 'Qtysk',
       'Qtyfk', 'Zcze', 'Zczelog', 'Treated', 'Treated_wide', 'Post', 'Rzrq',
       'Rzmr', 'Rzye', 'Rqmc', 'Rqye', 'Ltsz', 'Ltszlog', 'Zcfz', 'TobinQ',
       'Zmszb', 'Yszz', 'Roa', 'Hbl', 'Hsl', 'First', 'Jgtzz', 'Ddzb', 'Glccg',
       'Lzhy', 'Dshgm', 'Gqzhd', 'Two', 'Fxszz', 'Big4', 'Industry', 'Cqxz',
       'Qynl', 'Ssnx', 'Industry_d', 'HTobinQ', 'HHI', 'Dshgmlog'],
      dtype='object')

In [97]:
row_data["Fxslog"] = [np.log(1+i) if i >= 0 else np.nan for i in row_data["Fxszz"]]
row_data["Fxslog"]

0        4.189655
1        4.158883
2        4.094345
3        4.060443
4        3.988984
           ...   
38683         NaN
38684         NaN
38685         NaN
38686         NaN
38687    1.098612
Name: Fxslog, Length: 38688, dtype: float64

In [85]:
row_data["Rzmr_t"] = row_data["Rzmr"] / row_data["Ltsz"]
row_data["Rzye_t"] = row_data["Rzye"] / row_data["Ltsz"]
row_data["Rqmc_t"] = row_data["Rqmc"] / row_data["Ltsz"]
row_data["Rqye_t"] = row_data["Rqye"] / row_data["Ltsz"]

In [89]:
row_data["Time"] = row_data["Time"].astype(str)

In [99]:
row_data["Ssnxlog"] = [np.log(i + 1) if i >= 0 else np.nan for i in row_data["Ssnx"]]

In [91]:
os.chdir(r'D:\paper\rowdata\1')
row_data.to_csv("rowdata.csv", index=False)

In [101]:
row_data.dropna().to_csv("rowdata1.csv",index=False)

In [95]:
row_data.loc[:, ['Rzye', "Ltsz", "Rzmr_t"]]

,Rzye,Ltsz,Rzmr_t
0,0,1.046440e+11,0.000000
1,169648899,7.957169e+10,0.010167
2,557538781,7.231150e+10,0.047444
3,553109519,9.796418e+10,0.036554
4,2126345912,7.789111e+10,0.256302
...,...,...,...
38683,0,NaN,NaN
38684,0,NaN,NaN
38685,0,NaN,NaN
38686,0,NaN,NaN


In [102]:
q = row_data[row_data["Post"]==1]
q_ls = list(q[q["Rzmr"]==0].index)
row_data = row_data.drop(q_ls)

q = row_data[row_data["Post"]==0]
qqls = q[q["Rzmr"]!=0].index
row_data = row_data.drop(qqls)

row_data = row_data[row_data["Zcfz"]<1]

os.chdir(r'D:\paper\rowdata\ST')
ST = pd.read_csv("ST.csv", dtype={"Symbol": str})
ST_ls = []
for item in row_data["Code"]:
    if item in ST:
        ST_ls.append(1)
    else:
        ST_ls.append(0)

row_data["ST"] = ST_ls
row_data = row_data[row_data["ST"]==0]

row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog,ST
0,000002,2009-12-31,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,1.293807,0.075733,2.302585,4.189655,0.000000,0.000000,0.000000,0.000000,2.944439,0
1,000002,2010-12-31,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,1.059200,0.054805,2.397895,4.158883,0.010167,0.002132,0.000031,0.000004,2.995732,0
2,000002,2011-12-31,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.626153,0.063567,2.397895,4.094345,0.047444,0.007710,0.001231,0.000207,3.044522,0
3,000002,2012-12-31,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.668239,0.071580,2.397895,4.060443,0.036554,0.005646,0.004543,0.001130,3.091042,0
4,000002,2013-12-31,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.712191,0.076325,2.397895,3.988984,0.256302,0.027299,0.013770,0.000398,3.135494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38674,605598,2021-12-31,0,0,0,0,7.628710e+07,3.589567e+07,1.047497e+09,20.769670,...,0.233729,0.146733,1.609438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
38684,605599,2018-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,0.936766,0.092568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38685,605599,2019-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,1.110823,0.093244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38686,605599,2020-12-31,0,0,0,0,NaN,NaN,NaN,NaN,...,1.280476,0.131845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [103]:
row_data[row_data["ST"]==1]

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog,ST


In [105]:
row_data["Time"] = [i[:4] for i in row_data["Time"]]
row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog,ST
0,000002,2009,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,1.293807,0.075733,2.302585,4.189655,0.000000,0.000000,0.000000,0.000000,2.944439,0
1,000002,2010,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,1.059200,0.054805,2.397895,4.158883,0.010167,0.002132,0.000031,0.000004,2.995732,0
2,000002,2011,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.626153,0.063567,2.397895,4.094345,0.047444,0.007710,0.001231,0.000207,3.044522,0
3,000002,2012,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.668239,0.071580,2.397895,4.060443,0.036554,0.005646,0.004543,0.001130,3.091042,0
4,000002,2013,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.712191,0.076325,2.397895,3.988984,0.256302,0.027299,0.013770,0.000398,3.135494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38674,605598,2021,0,0,0,0,7.628710e+07,3.589567e+07,1.047497e+09,20.769670,...,0.233729,0.146733,1.609438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
38684,605599,2018,0,0,0,0,NaN,NaN,NaN,NaN,...,0.936766,0.092568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38685,605599,2019,0,0,0,0,NaN,NaN,NaN,NaN,...,1.110823,0.093244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38686,605599,2020,0,0,0,0,NaN,NaN,NaN,NaN,...,1.280476,0.131845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [121]:
row_data.dropna().iloc[:, 0:51].to_csv("row_data2.csv")

In [115]:
kk = row_data[row_data["Treated"]==0]
kk = kk[kk["Post"]==1]
kk = kk.index
row_data = row_data.drop(kk)
row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog,ST
0,000002,2009,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,1.293807,0.075733,2.302585,4.189655,0.000000,0.000000,0.000000,0.000000,2.944439,0
1,000002,2010,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,1.059200,0.054805,2.397895,4.158883,0.010167,0.002132,0.000031,0.000004,2.995732,0
2,000002,2011,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,0.626153,0.063567,2.397895,4.094345,0.047444,0.007710,0.001231,0.000207,3.044522,0
3,000002,2012,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,0.668239,0.071580,2.397895,4.060443,0.036554,0.005646,0.004543,0.001130,3.091042,0
4,000002,2013,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,0.712191,0.076325,2.397895,3.988984,0.256302,0.027299,0.013770,0.000398,3.135494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38674,605598,2021,0,0,0,0,7.628710e+07,3.589567e+07,1.047497e+09,20.769670,...,0.233729,0.146733,1.609438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
38684,605599,2018,0,0,0,0,NaN,NaN,NaN,NaN,...,0.936766,0.092568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38685,605599,2019,0,0,0,0,NaN,NaN,NaN,NaN,...,1.110823,0.093244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
38686,605599,2020,0,0,0,0,NaN,NaN,NaN,NaN,...,1.280476,0.131845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [120]:
os.chdir(r'D:\paper\rowdata\1')
row_data.dropna().iloc[:, 0:51].to_csv("row_data3.csv", index=False)

In [123]:
row_data = row_data.dropna().iloc[:, 0:51]
row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Industry_d,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog
0,000002,2009,0,0,0,0,4.049738e+10,5.208563e+09,6.150412e+10,24.842370,...,K70,1.293807,0.075733,2.302585,4.189655,0.000000,0.000000,0.000000,0.000000,2.944439
1,000002,2010,0,0,0,0,5.649274e+10,1.310024e+10,8.446798e+10,25.159638,...,K70,1.059200,0.054805,2.397895,4.158883,0.010167,0.002132,0.000031,0.000004,2.995732
2,000002,2011,0,0,0,0,6.590512e+10,2.305988e+10,9.172550e+10,25.242066,...,K70,0.626153,0.063567,2.397895,4.094345,0.047444,0.007710,0.001231,0.000207,3.044522
3,000002,2012,0,0,0,0,8.910113e+10,2.739208e+10,1.265448e+11,25.563862,...,K70,0.668239,0.071580,2.397895,4.060443,0.036554,0.005646,0.004543,0.001130,3.091042
4,000002,2013,0,0,0,0,1.301451e+11,4.418138e+10,1.636583e+11,25.821047,...,K70,0.712191,0.076325,2.397895,3.988984,0.256302,0.027299,0.013770,0.000398,3.135494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38440,605376,2021,0,0,0,0,4.211488e+08,5.020814e+07,1.592191e+09,21.188377,...,C33,2.633841,0.184755,2.197225,1.945910,0.000000,0.000000,0.000000,0.000000,0.693147
38453,605377,2021,0,0,0,0,3.266208e+08,1.432862e+07,3.131245e+09,21.864696,...,C22,1.490081,0.091153,2.197225,1.609438,0.000000,0.000000,0.000000,0.000000,0.693147
38479,605388,2021,0,1,0,1,2.775852e+08,1.695521e+06,1.921355e+09,21.376297,...,C15,8.403486,0.129214,2.302585,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147
38518,605399,2021,0,0,0,0,3.573900e+06,1.390492e+07,1.823445e+09,21.323993,...,C26,3.045007,0.036944,1.945910,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147


In [124]:
q = row_data[row_data["Rzmr"]!=0]
q[q["Rzmr_t"]==0]

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Industry_d,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog


In [126]:
row_data["Time"] = row_data["Time"].astype(int)

In [128]:
row_data = row_data[row_data["Time"]>2009]

In [129]:
row_data.columns

Index(['Code', 'Time', 'Vio', 'Vio_wide', 'NumVio', 'NumVio_wide', 'Qtysk',
       'Qtyfk', 'Zcze', 'Zczelog', 'Treated', 'Treated_wide', 'Post', 'Rzrq',
       'Rzmr', 'Rzye', 'Rqmc', 'Rqye', 'Ltsz', 'Ltszlog', 'Zcfz', 'TobinQ',
       'Zmszb', 'Yszz', 'Roa', 'Hbl', 'Hsl', 'First', 'Jgtzz', 'Ddzb', 'Glccg',
       'Lzhy', 'Dshgm', 'Gqzhd', 'Two', 'Fxszz', 'Big4', 'Industry', 'Cqxz',
       'Qynl', 'Ssnx', 'Industry_d', 'HTobinQ', 'HHI', 'Dshgmlog', 'Fxslog',
       'Rzmr_t', 'Rzye_t', 'Rqmc_t', 'Rqye_t', 'Ssnxlog'],
      dtype='object')

In [130]:
from scipy.stats.mstats import winsorize
row_data["Qtysk"] = winsorize(row_data["Qtysk"], limits=[0.01, 0.01])
row_data["Qtyfk"] = winsorize(row_data["Qtyfk"], limits=[0.01, 0.01])
row_data["Ltsz"] = winsorize(row_data["Ltsz"], limits=[0.01, 0.01])
row_data["Ltszlog"] = winsorize(row_data["Ltszlog"], limits=[0.01, 0.01])
row_data["Zczelog"] = winsorize(row_data["Zczelog"], limits=[0.01, 0.01])
row_data["Zcfz"] = winsorize(row_data["Zcfz"], limits=[0.01, 0.01])
row_data["Roa"] = winsorize(row_data["Roa"], limits=[0.01, 0.01])
row_data["TobinQ"] = winsorize(row_data["TobinQ"], limits=[0.01, 0.01])
row_data["Zmszb"] = winsorize(row_data["Zmszb"], limits=[0.01, 0.01])
row_data["Yszz"] = winsorize(row_data["Yszz"], limits=[0.01, 0.01])
row_data["Hbl"] = winsorize(row_data["Hbl"], limits=[0.01, 0.01])
row_data["Hsl"] = winsorize(row_data["Hsl"], limits=[0.01, 0.01])
row_data["First"] = winsorize(row_data["First"], limits=[0.01, 0.01])
row_data["Jgtzz"] = winsorize(row_data["Jgtzz"], limits=[0.01, 0.01])
row_data["Ddzb"] = winsorize(row_data["Ddzb"], limits=[0.01, 0.01])
row_data["Glccg"] = winsorize(row_data["Glccg"], limits=[0.01, 0.01])
row_data["Two"] = winsorize(row_data["Two"], limits=[0.01, 0.01])
row_data["Gqzhd"] = winsorize(row_data["Gqzhd"], limits=[0.01, 0.01])

C:\Users\S\AppData\Local\Temp\ipykernel_8060\3197219702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data["Qtysk"] = winsorize(row_data["Qtysk"], limits=[0.01, 0.01])
C:\Users\S\AppData\Local\Temp\ipykernel_8060\3197219702.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row_data["Qtyfk"] = winsorize(row_data["Qtyfk"], limits=[0.01, 0.01])
C:\Users\S\AppData\Local\Temp\ipykernel_8060\3197219702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [131]:
rzmr_q1 = row_data[row_data["Post"]==1]["Rzmr"].quantile(0.01, interpolation='lower')
rzmr_q2 = row_data[row_data["Post"]==1]["Rzmr"].quantile(0.99, interpolation='higher')
rzye_q1 = row_data[row_data["Post"]==1]["Rzye"].quantile(0.01, interpolation='lower')
rzye_q2 = row_data[row_data["Post"]==1]["Rzye"].quantile(0.99, interpolation='higher')
rqmc_q1 = row_data[row_data["Post"]==1]["Rqmc"].quantile(0.01, interpolation='lower')
rqmc_q2 = row_data[row_data["Post"]==1]["Rqmc"].quantile(0.99, interpolation='higher')
rqye_q1 = row_data[row_data["Post"]==1]["Rqye"].quantile(0.01, interpolation='lower')
rqye_q2 = row_data[row_data["Post"]==1]["Rqye"].quantile(0.99, interpolation='higher')

rzmr_t_q1 = row_data[row_data["Post"]==1]["Rzmr_t"].quantile(0.01, interpolation='lower')
rzmr_t_q2 = row_data[row_data["Post"]==1]["Rzmr_t"].quantile(0.99, interpolation='higher')
rzye_t_q1 = row_data[row_data["Post"]==1]["Rzye_t"].quantile(0.01, interpolation='lower')
rzye_t_q2 = row_data[row_data["Post"]==1]["Rzye_t"].quantile(0.99, interpolation='higher')
rqmc_t_q1 = row_data[row_data["Post"]==1]["Rqmc_t"].quantile(0.01, interpolation='lower')
rqmc_t_q2 = row_data[row_data["Post"]==1]["Rqmc_t"].quantile(0.99, interpolation='higher')
rqye_t_q1 = row_data[row_data["Post"]==1]["Rqye_t"].quantile(0.01, interpolation='lower')
rqye_t_q2 = row_data[row_data["Post"]==1]["Rqye_t"].quantile(0.99, interpolation='higher')

In [132]:
row_data.to_csv("rowdata4.csv",index=False)

In [133]:
row_data = pd.read_csv("rowdata4.csv", dtype={"Code": str, "Time": str})
row_data

,Code,Time,Vio,Vio_wide,NumVio,NumVio_wide,Qtysk,Qtyfk,Zcze,Zczelog,...,Industry_d,HTobinQ,HHI,Dshgmlog,Fxslog,Rzmr_t,Rzye_t,Rqmc_t,Rqye_t,Ssnxlog
0,000002,2010,0,0,0,0,2.499206e+10,1.310024e+10,8.446798e+10,25.159638,...,K70,1.059200,0.054805,2.397895,4.158883,0.010167,0.002132,0.000031,0.000004,2.995732
1,000002,2011,0,0,0,0,2.499206e+10,1.548560e+10,9.172550e+10,25.242066,...,K70,0.626153,0.063567,2.397895,4.094345,0.047444,0.007710,0.001231,0.000207,3.044522
2,000002,2012,0,0,0,0,2.499206e+10,1.548560e+10,1.265448e+11,25.563862,...,K70,0.668239,0.071580,2.397895,4.060443,0.036554,0.005646,0.004543,0.001130,3.091042
3,000002,2013,0,0,0,0,2.499206e+10,1.548560e+10,1.636583e+11,25.652501,...,K70,0.712191,0.076325,2.397895,3.988984,0.256302,0.027299,0.013770,0.000398,3.135494
4,000002,2014,0,0,0,0,2.499206e+10,1.548560e+10,1.746387e+11,25.652501,...,K70,0.845271,0.077816,2.397895,3.663562,0.232966,0.028760,0.007864,0.000519,3.178054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21714,605376,2021,0,0,0,0,4.211488e+08,5.020814e+07,1.592191e+09,21.188377,...,C33,2.633841,0.184755,2.197225,1.945910,0.000000,0.000000,0.000000,0.000000,0.693147
21715,605377,2021,0,0,0,0,3.266208e+08,1.432862e+07,3.131245e+09,21.864696,...,C22,1.490081,0.091153,2.197225,1.609438,0.000000,0.000000,0.000000,0.000000,0.693147
21716,605388,2021,0,1,0,1,2.775852e+08,1.695521e+06,1.921355e+09,21.376297,...,C15,8.403486,0.129214,2.302585,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147
21717,605399,2021,0,0,0,0,3.573900e+06,1.390492e+07,1.823445e+09,21.323993,...,C26,3.045007,0.036944,1.945910,0.693147,0.000000,0.000000,0.000000,0.000000,0.693147


In [134]:
rzmr = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rzmr.append(0)
    else:
        if qq["Rzmr"] <= rzmr_q1:
            rzmr.append(rzmr_q1)
        elif qq["Rzmr"] >= rzmr_q2:
            rzmr.append(rzmr_q2)
        else:
            rzmr.append(qq["Rzmr"])
row_data["Rzmr"] = rzmr

rzye = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rzye.append(0)
    else:
        if qq["Rzye"] <= rzye_q1:
            rzye.append(rzye_q1)
        elif qq["Rzye"] >= rzye_q2:
            rzye.append(rzye_q2)
        else:
            rzye.append(qq["Rzye"])
row_data["Rzye"] = rzye

rqmc = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rqmc.append(0)
    else:
        if qq["Rqmc"] <= rqmc_q1:
            rqmc.append(rqmc_q1)
        elif qq["Rqmc"] >= rqmc_q2:
            rqmc.append(rqmc_q2)
        else:
            rqmc.append(qq["Rqmc"])
row_data["Rqmc"] = rqmc

rqye = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rqye.append(0)
    else:
        if qq["Rqye"] <= rqye_q1:
            rqye.append(rqye_q1)
        elif qq["Rqye"] >= rqye_q2:
            rqye.append(rqye_q2)
        else:
            rqye.append(qq["Rqye"])
row_data["Rqye"] = rqye

rzmr_t = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rzmr_t.append(0)
    else:
        if qq["Rzmr_t"] <= rzmr_t_q1:
            rzmr_t.append(rzmr_t_q1)
        elif qq["Rzmr_t"] >= rzmr_t_q2:
            rzmr_t.append(rzmr_t_q2)
        else:
            rzmr_t.append(qq["Rzmr_t"])
row_data["Rzmr_t"] = rzmr_t

rzye_t = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rzye_t.append(0)
    else:
        if qq["Rzye_t"] <= rzye_t_q1:
            rzye_t.append(rzye_t_q1)
        elif qq["Rzye_t"] >= rzye_t_q2:
            rzye_t.append(rzye_t_q2)
        else:
            rzye_t.append(qq["Rzye_t"])
row_data["Rzye_t"] = rzye_t

rqmc_t = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rqmc_t.append(0)
    else:
        if qq["Rqmc_t"] <= rqmc_t_q1:
            rqmc_t.append(rqmc_t_q1)
        elif qq["Rqmc_t"] >= rqmc_t_q2:
            rqmc_t.append(rqmc_t_q2)
        else:
            rqmc_t.append(qq["Rqmc_t"])
row_data["Rqmc_t"] = rqmc_t

rqye_t = []
for i in range(21719):
    qq = row_data.iloc[i, :]
    if qq["Post"] == 0:
        rqye_t.append(0)
    else:
        if qq["Rqye_t"] <= rqye_t_q1:
            rqye_t.append(rqye_t_q1)
        elif qq["Rqye_t"] >= rqye_t_q2:
            rqye_t.append(rqye_t_q2)
        else:
            rqye_t.append(qq["Rqye_t"])
row_data["Rqye_t"] = rqye_t

In [143]:
row_data.dropna().to_csv("rowdata5.csv", index=False)

In [148]:
row_data.dropna().to_stata("rowdata5.dta", write_index=False)

In [147]:
row_data["Time"] = row_data["Time"].astype(int)

In [160]:
q = row_data[row_data["Time"]>2013]

q[q["Time"]<=2017].to_stata("rowdata7.dta", write_index=False)

In [158]:
row_data["Industry"] = row_data["Industry"].astype('category')

In [159]:
row_data.dropna().to_stata("rowdata11.dta", write_index=False)